# **Feature Extraction Methods: Balanced Data With Annotations and Augmentation**

- *Key Features*: [MFCCs, Mel-Spectrograms, Chroma Frequencies, RMS Power]
- *Key Manipulations*: [Varying Window Sizes, Normalization, Average Pooling (Compression), Filtering]
- *Process Assistence*: [Converting them to numpy arrays now, easy label access across features]
- *Conversion*: [To numpy arrays and pkl files]


In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import os
import time

# Libraries for audio
from IPython.display import Audio
import librosa
import librosa.display

# Training and Testing Split
from sklearn.model_selection import train_test_split

# for normalization & avgpooling features
import tensorflow as tf
# for preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
import random
import IPython.display as ipd

In [9]:
# Variabels to be reused
path =        'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/audio_files' 
npy_path =    'C:/Users/thato/Documents/Final-Year-Project/Dataset/test_audio_npy/' 
test_csv =    'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/test-not-annotated.csv' 
sr = 22050

In [10]:
test_data = pd.read_csv(test_csv)

In [11]:
test_data['species'].value_counts()

species
Charadrius alexandrinus      25
Himantopus himantopus        25
Botaurus stellaris           22
Tachybaptus ruficollis       22
Fulica atra                  20
Alcedo atthis                15
Gallinula chloropus          14
Circus aeruginosus           14
Acrocephalus scirpaceus      12
Ciconia ciconia              12
Motacilla flava              11
Ixobrychus minutus           11
Porphyrio porphyrio          11
Ardea purpurea               10
Coracias garrulus            10
Acrocephalus melanopogon      9
Dendrocopos minor             7
Anas platyrhynchos            7
Acrocephalus arundinaceus     5
Anas strepera                 2
Name: count, dtype: int64

In [12]:
test_data.head()

,Unnamed: 0,species,audio_name,duration,filename_npy
0,0,Acrocephalus scirpaceus,XC421435.mp3,9.247347,Acrocephalus scirpaceus/XC421435.npy
1,1,Tachybaptus ruficollis,XC459394.mp3,7.784490,Tachybaptus ruficollis/XC459394.npy
2,2,Fulica atra,XC543771.mp3,3.059002,Fulica atra/XC543771.npy
3,3,Coracias garrulus,XC318427.mp3,760.058776,Coracias garrulus/XC318427.npy
4,4,Circus aeruginosus,XC100621.mp3,18.840000,Circus aeruginosus/XC100621.npy


# **Creating a class to do the extraction**

In [16]:
class Extraction:
  def __init__(self, test_df, window_size, overlap=0.5, npy_path=npy_path, sr=sr, n_mels=128, n_mfcc=20, n_chroma=12, n_cqt=84, hoplength=256, features=['mfcc'], normalize=True, avgpool=False, augment=False):
    """
    Instantiate the Extraction class to extract features.

    Parameters:
      sr (int): Sample rate of the audio files.
      n_mfccs (int): Number of MFCCs to extract.
      n_mels (int): Number of Mel bands to extract.
      n_chroma (int): Number of chroma bins to use.
      n_cqt (int): Number of CQT bins to use.
      features (list): List of features to extract.
        accepted features: 'mfcc', 'chroma', 'cqt', 'melspectrogram'.
      normalize (bool): Whether to normalize the features.
      avgpool (bool): Whether to avgpool the features.
    """
    
    self.test_df = test_df
    self.npy_path = npy_path
    self.window_size = window_size
    self.overlap = overlap
    self.sr = sr
    self.n_mels = n_mels
    self.n_mfcc = n_mfcc
    self.n_chroma = n_chroma
    self.n_cqt = n_cqt
    self.hoplength = hoplength
    
     # confirm features have been specified
    assert len(features) != 0, "Must Specify At Least One Feature In The Form Of A List."
    self.features = features

    self.accepted_feature = ['mfcc', 'chroma', 'cqt', 'melspectrogram']
    for feature in self.features:
      assert feature in self.accepted_feature, f"{feature} is not an accepted feature, only 'mfcc', 'chroma', 'cqt', 'melspectrogram' are accepted features."

    self.normalize = normalize
    self.avgpool = avgpool
    self.augment = augment

    print(f"Test DataFrame shape: {test_df.shape}")

    # extract train and val labels and features
    self.test_y, self.test_features, self.test_ids = self.feature_extraction(self.test_df, window_size=self.window_size)

    # process the features by average pooling
    self.test_features_2D, self.test_features_1D,  = self.process_features(self.test_features)

  def random_augmentation(self, audio):
        """
        Apply random augmentation to the audio: pitch shift, time stretch, or add noise.
        """
        aug_type = np.random.choice(['pitch_up', 'pitch_down', 'noise', 'stretch', 'shrink'])

        if aug_type == 'pitch_up':
            # Random pitch up (between 1 and 4 semitones)
            pitch_shift = np.random.uniform(1, 4)
            audio = librosa.effects.pitch_shift(audio, sr=self.sr, n_steps=pitch_shift)
        elif aug_type == 'pitch_down':
            # Random pitch down (between -1 and -4 semitones)
            pitch_shift = np.random.uniform(-4, -1)
            audio = librosa.effects.pitch_shift(audio, sr=self.sr, n_steps=pitch_shift)
        elif aug_type == 'noise':
            # Add random noise at a fraction of the original audio volume
            noise = self.generate_pink_noise(len(audio))
            audio = audio + 0.01 * np.random.uniform(0.2, 0.5) * np.max(audio) * noise
        elif aug_type == 'stretch':
            # Randomly stretch audio (speed up)
            stretch_factor = np.random.uniform(1.1, 1.5)
            audio = librosa.effects.time_stretch(audio, rate=stretch_factor)
        elif aug_type == 'shrink':
            # Randomly shrink audio (slow down)
            stretch_factor = np.random.uniform(0.8, 0.9)
            audio = librosa.effects.time_stretch(audio, rate=stretch_factor)

        return audio

  def normalize_audio(self, audio):
    return (audio - np.min(audio)) / (np.max(audio) - np.min(audio))
  
  def generate_pink_noise(self, num_samples):
    white_noise = np.random.randn(num_samples)
    
    # Apply a filter to convert white noise into pink noise (1/f noise)
    X = np.fft.rfft(white_noise)
    S = np.arange(1, len(X) + 1)  # Frequency scaling
    pink_noise = np.fft.irfft(X / S)

    if len(pink_noise) < num_samples:
        # Pad with zeros if the length is less than num_samples
        pink_noise = np.pad(pink_noise, (0, num_samples - len(pink_noise)), mode='constant')
    elif len(pink_noise) > num_samples:
        # Trim if necessary
        pink_noise = pink_noise[:num_samples]
    
    return self.normalize_audio(pink_noise)
  
  def pad_with_noise(self, audio_data, window_length, window_samples):
    current_length = librosa.get_duration(y=audio_data, sr=self.sr)

    if current_length > window_length:
        return audio_data
    
    target_length_samples = int(window_length * sr) 
    current_length_samples = window_samples
    padding_length_samples = target_length_samples - current_length_samples

    assert target_length_samples == (current_length_samples+padding_length_samples)
    
    # Generate pink noise to pad with
    pink_noise = self.generate_pink_noise(padding_length_samples)
    padded_audio = np.concatenate([audio_data, pink_noise])
    # if len(padded_audio) < target_length_samples:
    #     padded_audio = np.append(padded_audio, self.generate_pink_noise(1))

    assert target_length_samples == len(padded_audio)
    
    return padded_audio
  
  def avg_pooling_keras(self, feature):
    # Clear the previous Keras session
    tf.keras.backend.clear_session()

    # Define the input shape based on features
    input_shape = feature.shape[1:]  # (n_mels, time_steps)

    # Create the Keras model for average pooling
    inputs = tf.keras.layers.Input(shape=input_shape)
    pooled = tf.keras.layers.GlobalAveragePooling1D()(inputs)
    pooling_model = tf.keras.models.Model(inputs=inputs, outputs=pooled)

    # Perform pooling using the model
    pooled_features = pooling_model.predict(feature)

    return pooled_features

#-------------------------Feature Extraction---------------------------------------
  def extract_mfcc(self, window):
    mfcc = librosa.feature.mfcc(y=window, sr=self.sr, n_mfcc=self.n_mfcc, hop_length=self.hoplength)
    if self.normalize:
      return librosa.util.normalize(mfcc)
    else:
      return mfcc


  def extract_chroma(self, window):
    chroma = librosa.feature.chroma_stft(y=window, sr=self.sr, n_chroma=self.n_chroma, hop_length=self.hoplength)
    if self.normalize:
      return librosa.util.normalize(chroma)
    else:
      return chroma
   

  def extract_cqt(self, window):
    cqt = librosa.cqt(y=window, sr=sr, hop_length=self.hoplength, n_bins=self.n_cqt)
    cqt_db = librosa.amplitude_to_db(np.abs(cqt), ref=np.max)
    return cqt_db

  def extract_melspectrogram(self, window):
    mel = librosa.feature.melspectrogram(y=window, sr=self.sr, n_mels=self.n_mels, hop_length=self.hoplength)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    if self.normalize:
      return librosa.util.normalize(mel_db)
    else:
      return mel_db
    
  def avgpooling(self, train_X, n_time, n_features):
    """
    Average pooling the train and val features.

    Parameters:
      train_X (npy): Training feature array of shape (batch_size, n_features, n_time)
      val_X (npy): Validation feature array of shape (batch_size, n_features, n_time)
      n_time (int): Time axis
      n_features (int): Feature axis

    Returns:
      train_X (npy): Avgpooled training feature array of shape (batch_size, n_features)
      val_X (npy): Avgpooled validation feature array of shape (batch_size, n_features)
    """
    # Clear the Keras session
    tf.keras.backend.clear_session()
    
    # Create the Keras input layer with shape (n_features, n_time)
    input_layer = tf.keras.layers.Input(shape=(n_features, n_time))
    
    # Apply average pooling over the time axis (axis=-1) to reduce n_time
    avg_pool = tf.keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1))(input_layer)
    
    # Build the model
    pooling_model = tf.keras.models.Model(inputs=input_layer, outputs=avg_pool)

    # Use the model to apply average pooling on the training and validation features
    train_X = pooling_model.predict(train_X)

    return train_X

    
  def process_features(self, train_features_dict):
    train_copy = train_features_dict.copy()
    for each in train_copy.keys():
      
      if each == 'mfcc':
        n_features=self.n_mfcc
      elif each == 'chroma':
        n_features=self.n_chroma
      elif each == 'cqt':
        n_features=self.n_cqt
      elif each == 'melspectrogram':
        n_features=self.n_mels
      
      train_feature = train_copy[each]

      if self.avgpool:
        train_copy[each] = self.avgpooling(train_feature, n_time=train_feature.shape[2], n_features=n_features)
      else:
        train_copy[each] = train_copy[each]
    
    return train_features_dict, train_copy
      

  def feature_extraction(self, dataframe, window_size):
    y = [] # To hold the labels
    ids = []
    features_dict = {item: [] for item in self.features} # Create a key for each feature listed
    print(f"Number of rows in dataframe: {len(dataframe)}")
    for _, row in tqdm(dataframe.iterrows(), desc="Processing data", total=len(dataframe)):
          label = row['species']
          file_path =  os.path.join(self.npy_path, row['filename_npy'])
          id = row['audio_name']
          

          # print(f"Processing file: {file_path}")

          try:
            
              audio = np.load(file_path)
          except FileNotFoundError:
              print(f"File not found: {file_path}")
              continue


          sample = audio.copy()

          if len(sample) < 512:
                continue

          sample = self.normalize_audio(sample)

          sample = self.pad_with_noise(sample, window_length=self.window_size, window_samples=len(sample))
          

          window_samples = int(window_size * self.sr)
          hop_samples = int(window_samples * (1 - self.overlap))  # For overlapping

          # Break the audio into windows with the specified overlap
          audio_windows = librosa.util.frame(sample, frame_length=window_samples, hop_length=hop_samples).T
          
          
          # display(label)
          
          for _, window in enumerate(audio_windows):
              
              y.append(label)
              ids.append(id)

              if len(window) < window_samples:
                  if len(window) < 512*2:
                     continue
                  else:
                      window = self.pad_with_noise(window, window_length=window_size)
              
              # Feature Extraction FR --------------------------------------------------------------------
              # dynatically call the extract_x function to extract the listed features
              for feature in self.features:
                extract = f"extract_{feature}"
                if hasattr(self, extract) and callable(func := getattr(self, extract)):
                  features_dict[feature].append(func(window))

          # cast lists to np arrays
    for each in features_dict.keys():
              features_dict[each] = np.array(features_dict[each])

    y = np.array(y)
    ids = np.array(ids)

          # If not using average pooling, return resized features
    return y, features_dict, ids

## **Window Size = 3s**

In [14]:
features_list = ['melspectrogram', 'mfcc', 'chroma', 'cqt']

In [17]:
features = Extraction(test_data,
                      window_size=3,
                      features=features_list,
                      avgpool=True
                      )

Test DataFrame shape: (264, 5)
Number of rows in dataframe: 264


Processing data:   0%|          | 0/264 [00:00<?, ?it/s]c:\Users\thato\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
Processing data: 100%|██████████| 264/264 [08:33<00:00,  1.94s/it]



230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 665us/step
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 672us/step
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 985us/step


In [18]:
test_y = features.test_y

display(test_y.shape)

(7349,)

In [19]:
# Avgpooled Features
test_features_1D = features.test_features_1D
for key in test_features_1D.keys():
  display(key)
  display(test_features_1D[key].shape)

'melspectrogram'

(7349, 128)

'mfcc'

(7349, 20)

'chroma'

(7349, 12)

'cqt'

(7349, 84)

In [20]:
# Not avgpooled Features
test_features_2D = features.test_features_2D
for key in test_features_2D.keys():
  display(key)
  display(test_features_2D[key].shape)

'melspectrogram'

(7349, 128, 259)

'mfcc'

(7349, 20, 259)

'chroma'

(7349, 12, 259)

'cqt'

(7349, 84, 259)

In [21]:
test_ids = features.test_ids

### Encode Classes

In [22]:
label_encoder = LabelEncoder().fit(test_y)
test_y_encoded = label_encoder.transform(test_y)

classes = list(label_encoder.inverse_transform([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]))
print("Encoded classes:", classes)
print("Encoded training labels:", test_y_encoded)

Encoded classes: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus', 'Alcedo atthis', 'Anas platyrhynchos', 'Anas strepera', 'Ardea purpurea', 'Botaurus stellaris', 'Charadrius alexandrinus', 'Ciconia ciconia', 'Circus aeruginosus', 'Coracias garrulus', 'Dendrocopos minor', 'Fulica atra', 'Gallinula chloropus', 'Himantopus himantopus', 'Ixobrychus minutus', 'Motacilla flava', 'Porphyrio porphyrio', 'Tachybaptus ruficollis']
Encoded training labels: [2 2 2 ... 3 3 3]


In [23]:
# Avg Pooled

test_features_1D['label'] = test_y_encoded


test_features_1D['id'] = test_ids


# Not Avg Pooled

test_features_2D['label'] = test_y_encoded

test_features_2D['id'] = test_ids


In [24]:
merged_dict_1D = {'test': test_features_1D}
merged_dict_1D

{'test': {'melspectrogram': array([[-0.0176525 , -0.97828585, -0.98017514, ..., -0.71472144,
          -0.747796  , -0.8750666 ],
         [-0.01765439, -0.9782942 , -0.98016405, ..., -0.7101959 ,
          -0.74066454, -0.8719466 ],
         [-0.01765663, -0.9784361 , -0.98033446, ..., -0.7191713 ,
          -0.7481973 , -0.87630737],
         ...,
         [-0.0203373 , -0.9024343 , -0.88190603, ..., -0.8073667 ,
          -0.8481265 , -0.9984192 ],
         [-0.01999963, -0.8998117 , -0.8703364 , ..., -0.8076148 ,
          -0.84703326, -0.99737525],
         [-0.01995196, -0.9041279 , -0.86233217, ..., -0.80547255,
          -0.8468942 , -0.99675184]], dtype=float32),
  'mfcc': array([[-0.99985766, -0.26917735, -0.1320615 , ...,  0.03152308,
           0.02190971,  0.04245044],
         [-0.9994832 , -0.3066085 , -0.15518537, ...,  0.03022101,
           0.0339629 ,  0.04487871],
         [-0.9997456 , -0.29917315, -0.14050432, ...,  0.03299716,
           0.03303996,  0.04636334],

In [25]:
merged_dict_2D = {'test': test_features_2D}
merged_dict_2D

{'test': {'melspectrogram': array([[[-0.01736392,  0.        , -0.00813095, ..., -0.01304301,
           -0.00132603, -0.00622145],
          [-0.1938493 , -0.21230054, -0.26985499, ..., -0.30557457,
           -0.22740984, -0.19391967],
          [-0.26834923, -0.28665653, -0.34289044, ..., -0.37502521,
           -0.29998928, -0.26736379],
          ...,
          [-0.99895918, -0.99854666, -0.9918586 , ..., -0.89447725,
           -0.95579219, -0.97648239],
          [-0.99957073, -0.99912   , -0.99232167, ..., -0.91377246,
           -0.97081757, -0.99212426],
          [-1.        , -1.        , -1.        , ..., -1.        ,
           -1.        , -1.        ]],
  
         [[-0.01770214,  0.        , -0.00831963, ..., -0.01256057,
           -0.00124793, -0.00581391],
          [-0.19749348, -0.21654767, -0.27626368, ..., -0.29428038,
           -0.21418518, -0.18125534],
          [-0.27339751, -0.29239982, -0.3510446 , ..., -0.36115357,
           -0.28253412, -0.24989378],
 

### Save the merged dictionary to a pkl

In [26]:
# Avrg Pooled
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/NotAnnotatedTest/test_features_3s_all_1D.pkl', 'wb') as file:
  pickle.dump(merged_dict_1D, file)
del file

In [27]:
# Avrg Pooled
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/NotAnnotatedTest/test_features_3s_all_2D.pkl', 'wb') as file:
  pickle.dump(merged_dict_2D, file)
del file